In [2]:
import openai

In [3]:
from openai import OpenAI

In [3]:
client = OpenAI()

In [4]:
import os

In [139]:
#from dotenv import load_dotenv, find_dotenv
#_ = load_dotenv(find_dotenv())
api_key = os.getenv("OPENAI_API_KEY")

#openai.api_key=

In [10]:
openai.api_key=api_key

In [13]:
response = client.chat.completions.create(model='gpt-4o', messages=[{"role":"user", "content":"is it late to join the course"}])

In [14]:
response.choices[0].message.content

"The answer to whether it's late to join a course depends on a variety of factors:\n\n1. **Course Start Date**: If the course has just started or is still in the early weeks, it might be possible to catch up with some extra effort.\n2. **Course Duration**: For shorter courses, joining late might be more challenging compared to longer courses where there is more time to catch up.\n3. **Instructor Policies**: Some instructors are flexible and may allow late enrollments with additional support, while others may have strict deadlines.\n4. **Course Format**: Online and self-paced courses might offer more flexibility for late joiners compared to in-person or synchronous online courses.\n5. **Prerequisites and Initial Content**: If there are foundational concepts covered at the beginning of the course, missing these might make it difficult to understand later material.\n6. **Resources Availability**: Check if recorded lectures, notes, and other materials are available to review the missed con

In [6]:
import minsearch

In [7]:
import json

In [8]:
with open ('documents.json', 'rt') as f_in:
    doc_raw = json.load(f_in)
    

In [9]:
documents = []
for course in doc_raw:
    for doc in course['documents']:
        doc['course'] = course['course']
        documents.append(doc)


In [23]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [10]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [11]:
#siilar to select * where course = ''data-engineering-zoomcamp''

In [35]:
question='the course has already started, can I still enroll'

In [13]:
#fit the serach withthe documents
index.fit(documents)

In [36]:
boost = {'question':3.0, 'section':.5}
knowledge_results = index.search(query=question, filter_dict={'course':'data-engineering-zoomcamp'},
                       boost_dict=boost, num_results=5)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [37]:
knowledge_results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [38]:
from openai import OpenAI

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

response = client.chat.completions.create(
    model="gpt-3.5-turbo",    
    messages=[{"role": "user", "content": question}]
)
print(response.choices[0].message.content)

It depends on the policy of the institution offering the course. Some institutions may allow late enrollment with permission from the instructor or department, while others may have strict deadlines for enrollment. It's best to contact the institution or the course instructor directly to inquire about late enrollment options.


In [39]:
context=''
for doc in knowledge_results:
    context = context + f"section: {doc['section']}\nquestioin:{doc['question']}\nanswer:{doc['text']}\n"
    

In [40]:
context

"section: General course-related questions\nquestioin:Course - Can I still join the course after the start date?\nanswer:Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.\nsection: General course-related questions\nquestioin:Course - Can I follow the course after it finishes?\nanswer:Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.\nsection: General course-related questions\nquestioin:Course - When will the course start?\nanswer:The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will st

In [41]:
prompt = f"""
You're a course teaching assistant. Answer the user QUESTION based on CONTEXT - the documents retrieved from our FAQ database. 
Only use the facts from the CONTEXT. If the CONTEXT doesn't contan the answer, return "NONE"

QUESTION: {question}

CONTEXT:

{context}
""".strip()

In [68]:
def search(query):
    boost = {'question':3.0, 'section':.5}
    knowledge_results = index.search(query=query, filter_dict={'course':'data-engineering-zoomcamp'},
                       boost_dict=boost, num_results=5)
    return knowledge_results


In [61]:
def create_prompt(knowledge_results, query):
    context=''
    for doc in knowledge_results:
        context = context + f"section: {doc['section']}\nquestioin:{doc['question']}\nanswer:{doc['text']}\n"
    prompt = f"""
    You're a course teaching assistant. Answer the user QUESTION based on CONTEXT - the documents retrieved from our FAQ database. 
    Only use the facts from the CONTEXT. If the CONTEXT doesn't contan the answer, return "NONE"
                
    QUESTION: {question}
                
    CONTEXT:
                
    {context}
    """.strip()
    prompt = prompt.format(question=query, context=context)
    return prompt

In [62]:
prompt = create_prompt(knowledge_results, question)
prompt

'You\'re a course teaching assistant. Answer the user QUESTION based on CONTEXT - the documents retrieved from our FAQ database. \n    Only use the facts from the CONTEXT. If the CONTEXT doesn\'t contan the answer, return "NONE"\n                \n    QUESTION: the course has already started, can I still enroll\n                \n    CONTEXT:\n                \n    section: Module 6: streaming with kafka\nquestioin:Java Kafka: How to run producer/consumer/kstreams/etc in terminal\nanswer:In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\nsection: Module 6: streaming with kafka\nquestioin:Module “kafka” not found when trying to run producer.py\nanswer:Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.\nTo create a virtual env and install packages (run only once)\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\nT

In [80]:
def llm(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=[{"role": "user", "content": prompt}]
    )
    return (response.choices[0].message.content)

In [81]:
def rag(query):
    knowledge_results = search(query)
    prompt = create_prompt(knowledge_results, query)
    llm(prompt)


In [82]:
query = 'How to enroll in a course?'
print(rag(query))

None


In [83]:
from elasticsearch import Elasticsearch

In [84]:
es = Elasticsearch("http://localhost:9200")
es.info()

ObjectApiResponse({'name': '2cced2761d18', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'yOc1-ouATVC_GRIM7X1OTQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [85]:

#create index
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
response = es.indices.create(index=index_name, body=index_settings)

response

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [86]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [88]:
from tqdm.auto import tqdm

In [89]:
#index
for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [122]:
user_question = "How do I execute a command in a running docker container?"
#question^3 questio is three times more important
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [110]:
import requests

In [118]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [119]:
#index
for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [123]:
response = es.search(index=index_name, body=search_query)
max_score = 0
for hit in response['hits']['hits']:
    doc = hit['_source']
    if hit['_score'] > max_score:
        max_score = hit['_score']
    print(f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n")
print('max score', max_score)

Section: 5. Deploying Machine Learning Models
Question: How do I debug a docker container?
Answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)


Section: 5. Deploying Machine Learning Models
Question: How do I debug a docker container?
Answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)


Section: 5. Deploying Machine Learning Models
Question: How do I debug a docker container?
Answer: Launch the container image in interactive mode and overriding t

In [93]:
#response['hits']['hits']

In [101]:
 response['hits']['hits'][0]['_score']

75.54128

In [130]:
def create_prompt_elastic(es_results, query):
    context=''
    for doc in es_results['hits']['hits']:
        doc = hit['_source']
        context = context + f"section: {doc['section']}\nquestioin:{doc['question']}\nanswer:{doc['text']}\n"
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
    prompt = prompt_template.format(question=query, context=context)
    return prompt

In [138]:
question = "How do I execute a command in a running docker container?" 
prompt = create_prompt_elastic(response, question)
print('Question 5. Prompt length', len(create_prompt_elastic(response, question)))

Question 5. Prompt length 1492


In [132]:
pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 870.5 kB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.4 MB/s eta 0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 17.5 MB/s eta 0:00:00m eta 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [134]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [140]:
#Q6
print('Question 6. Number of tokens ', len(encoding.encode(prompt)))

Question 6. Number of tokens  337
